In [11]:
import numpy as np
import pandas as pd

In [2]:
import finance_fns.stock_handle as stock_handle

In [4]:
stock_code = stock_handle.find_code('NAVER')
stock_code

'035420'

In [5]:
df = stock_handle.get_close_df(stock_code, start_date='2024')
df

,Close
Date,
2024-01-02,227500
2024-01-03,222000
2024-01-04,223000
2024-01-05,220500
2024-01-08,226500
...,...
2024-12-09,201500
2024-12-10,209000
2024-12-11,218000


In [6]:
adder = stock_handle.add_to_df(df)

In [8]:
adder('change', lambda df: df['Close'] - df['Close'].shift(1))

,Close,change
Date,,
2024-01-02,227500,NaN
2024-01-03,222000,-5500.0
2024-01-04,223000,1000.0
2024-01-05,220500,-2500.0
2024-01-08,226500,6000.0
...,...,...
2024-12-09,201500,-3000.0
2024-12-10,209000,7500.0
2024-12-11,218000,9000.0


In [9]:
adder('gain', lambda df: np.where(df['change']>=0, df['change'], 0))
adder('loss', lambda df: np.where(df['change']<0, df['change'], 0))

,Close,change,gain,loss
Date,,,,
2024-01-02,227500,NaN,0.0,0.0
2024-01-03,222000,-5500.0,0.0,-5500.0
2024-01-04,223000,1000.0,1000.0,0.0
2024-01-05,220500,-2500.0,0.0,-2500.0
2024-01-08,226500,6000.0,6000.0,0.0
...,...,...,...,...
2024-12-09,201500,-3000.0,0.0,-3000.0
2024-12-10,209000,7500.0,7500.0,0.0
2024-12-11,218000,9000.0,9000.0,0.0


In [10]:
adder('avg_gain', lambda df: df['gain'].rolling(14).mean())
adder('avg_loss', lambda df: df['loss'].rolling(14).mean())

,Close,change,gain,loss,avg_gain,avg_loss
Date,,,,,,
2024-01-02,227500,NaN,0.0,0.0,NaN,NaN
2024-01-03,222000,-5500.0,0.0,-5500.0,NaN,NaN
2024-01-04,223000,1000.0,1000.0,0.0,NaN,NaN
2024-01-05,220500,-2500.0,0.0,-2500.0,NaN,NaN
2024-01-08,226500,6000.0,6000.0,0.0,NaN,NaN
...,...,...,...,...,...,...
2024-12-09,201500,-3000.0,0.0,-3000.0,1742.857143,-1135.714286
2024-12-10,209000,7500.0,7500.0,0.0,2271.428571,-1135.714286
2024-12-11,218000,9000.0,9000.0,0.0,2914.285714,-892.857143


In [13]:
def show_rsi(stock_code, rsi_period = 14) -> pd.DataFrame:
    df_close = stock_handle.get_close_df(stock_code, start_date='2024')
    add_index = stock_handle.add_to_df(df_close)
    add_index('change', lambda df: df['Close'] - df['Close'].shift(1))
    add_index('gain', lambda df: np.where(df['change']>=0, df['change'], 0))
    add_index('loss', lambda df: np.where(df['change']<0, -df['change'], 0))
    add_index('avg_gain', lambda df: df['gain'].rolling(rsi_period).mean())
    add_index('avg_loss', lambda df: df['loss'].rolling(rsi_period).mean())
    final_df = add_index('rsi', lambda df: 100 - (100 / (1 + df['avg_gain']/df['avg_loss'])))
    return final_df[['rsi']]

show_rsi(stock_code)


,rsi
Date,
2024-01-02,NaN
2024-01-03,NaN
2024-01-04,NaN
2024-01-05,NaN
2024-01-08,NaN
...,...
2024-12-09,60.545906
2024-12-10,66.666667
2024-12-11,76.547842
